<a href="https://colab.research.google.com/github/ivansst773/Aprendizaje_de_Maquina/blob/main/Parcial_2_TAM_2025_1/Parte_Practica/Parcial_2_TAM_2025_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar dependencias en Colab
!pip install dash jupyter-dash umap-learn plotly scikit-learn tensorflow

In [3]:
# Importar bibliotecas
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import umap
import base64
from io import BytesIO
from PIL import Image

2025-07-21 22:48:01.866123: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-21 22:48:02.270736: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-21 22:48:02.395075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753138083.005824   36626 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753138083.054735   36626 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753138083.713901   36626 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [4]:
# Cargar y preprocesar datos USPS
digits = load_digits()
X, y = digits.data / 255.0, digits.target  # Normalizar
images = digits.images  # Para superponer imágenes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Proyecciones PCA y UMAP
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

umap_model = umap.UMAP(n_components=2, n_neighbors=15, random_state=42)
X_umap = umap_model.fit_transform(X)

/workspaces/Aprendizaje_de_Maquina/.venv/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [6]:
def train_classifiers(X_train, X_test, y_train, y_test):
    # 1. LogisticRegression
    lr = LogisticRegression(C=1.0, multi_class='multinomial', solver='lbfgs', max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    report_lr = classification_report(y_test, y_pred_lr, output_dict=True)

    # 2. RandomForestClassifier
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    report_rf = classification_report(y_test, y_pred_rf, output_dict=True)

    # 3. CNN
    X_train_cnn = X_train.reshape(-1, 8, 8, 1)
    X_test_cnn = X_test.reshape(-1, 8, 8, 1)
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, verbose=0)
    y_pred_cnn = model.predict(X_test_cnn).argmax(axis=1)
    report_cnn = classification_report(y_test, y_pred_cnn, output_dict=True)

    # Guardar el modelo CNN
    model.save("modelo_dl.h5")

    return lr, report_lr, rf, report_rf, model, report_cnn

lr, report_lr, rf, report_rf, cnn_model, report_cnn = train_classifiers(X_train, X_test, y_train, y_test)

/workspaces/Aprendizaje_de_Maquina/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/workspaces/Aprendizaje_de_Maquina/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-21 22:48:34.384498: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
